# Data science project for the 13/04/2019 event
## purpose of the project

Le but de ce projet est de scraper les donnees pour faire une presentation


In [2]:
# importing modules for our work
#importing numpy
import numpy as np
# importing matplotlib
%matplotlib inline
import matplotlib.pyplot as plt
# importing pandas
import pandas as pd
# importing request
import requests
import os, os.path
# importing beautifull soup
from bs4 import BeautifulSoup
from urllib.parse import urljoin, urlparse
import re

In [ ]:
url = 'https://www.jumia.cm/smartphones/'
#pages_to_crawl = 24
#number of pages that will be scrap
pages_to_crawl = 25
r= requests.get(url)
#soup = BeautifulSoup(r.text, 'html.parser')
''' the for look bellow help get the link of each item and we can open each to investigate more in porder to get insight
from the data store in it'''
#links = soup.find_all(class_="link")
# getting all the links and putting them in an array
links =[]

# difining a dictionarry of data that need to be scrapped

data ={'name':[],
      'seller':[],
      'price':[],
      'number-of-items-sold': [],
      'ram': [],
      'color': [],
      'screen-size': [],
       'rom': [],
       'front-camera':[],
       'rear-camera':[],
       'number_of_sims':[]
      }

# scarping the pages
for p in range(1, pages_to_crawl+1):
    print('Scraping page:', p)
    r = requests.get(url, params={'page' : p})
    soup = BeautifulSoup(r.text, 'html.parser')
    for found  in soup.findAll('a', attrs={'class':'link', 'href': re.compile("^https://")}):
        links.append(found.get('href'))
##print (links)

for i in range(0, len(links)):
    item_link = requests.get(links[i])
    item_link_soup =BeautifulSoup(item_link.text, 'html.parser')
    
    ''' make sure that the seller name exist since this is the brand of the phone , make sure that the brand name is not empty and also that the brand name is different from rock since we have realize that rock is not a phone brand
    and also the type generic is the type of every other item not phones'''
    if item_link_soup.find('div', {'class':'sub-title'}).find('a') is not None and item_link_soup.find('div', {'class':'sub-title'}).find('a').get_text(strip=True).lower() !='generic' and  item_link_soup.find('div', {'class':'sub-title'}).find('a').get_text(strip=True).lower() !='rock':
        vendeur = item_link_soup.find('div', {'class':'sub-title'}).find('a').get_text(strip=True)
        data['seller'].append(vendeur)
        name = item_link_soup.find('h1', {'class': 'title' }).get_text(strip=True)
        if('dual sim' in name.lower()):
            data['number_of_sims'].append(2)
        else:
            data['number_of_sims'].append(1)
        data['name'].append(name)
        #getting the price of items
        prise = item_link_soup.find('span', {'dir':'ltr'})
        prize = prise.get_text(strip=True).replace(',', '')
        data['price'].append(float(prize))
        
        # getting the numbers of items sold
        if item_link_soup.find('span', {'class': 'text color-default bold -prxs -inline-block -bold'}) is not None:
            vendu = item_link_soup.find('span', {'class': 'text color-default bold -prxs -inline-block -bold'}).get_text(strip=True)
            if vendu.endswith('+'):
                vendu1 = vendu.replace('+', '')
                vendu2 = vendu1.replace(',', '')
                vendu_final = float(vendu2) + 1
                data['number-of-items-sold'].append(vendu_final)
            # here due to the fact that jumia.cm doesn't give the exact number sold, we add 1 if its said to be more than or sousract if to be less than.
            elif vendu.startswith('<'):
                vendu1 = vendu.replace('<', '')
                vendu2 = vendu1.replace(',', '')
                vendu_final = float(vendu2) - 1
                data['number-of-items-sold'].append(vendu_final)
            else :
                data['number-of-items-sold'].append(float(0.0))
        else:
            ok = 'not know'
            
        #getting the color of the phone
        
        for caracteristiques in item_link_soup.find_all('div', {'class':'osh-row '}):
            value = caracteristiques.find_all('div', {'class': 'osh-col'})
            #print(value)
            for value1 in value:
            #print(value1)
                if 'couleur principale' in value[0].get_text(strip=True).lower():
                    couleur = value[1].get_text(strip=True).lower()
                    #print(couleur)
                    data['color'].append(value[1].get_text(strip=True).lower())
                    #print(value)
                elif 'taille de' in value[0].get_text(strip=True).lower():
                     data['screen-size'].append(value[1].get_text(strip=True).lower())
                elif 'ram syst' in value[0].get_text(strip=True).lower():
                    data['ram'].append(float(value[1].get_text(strip=True).lower()))
                elif 'capacit' in value[0].get_text(strip=True).lower():
                     data['rom'].append(float(value[1].get_text(strip=True).lower()))
                else:
                    a = 'ok'
        # getting the camera info
        if(len(item_link_soup.find_all('ul')[4].find_all('li'))>2):
            cam1 = str(item_link_soup.find_all('ul')[4].find_all('li')[2].get_text(strip= True)).lower()
            if('caméra' in cam1 or 'photo' in cam1):
                arr1 = [float(s) for s in re.findall(r'-?\d+\.?\d*', cam1)]
                arr2 = [x for x in arr1 if x != 0.0]
                if len(arr2) == 2:
                    data['rear-camera'].append(arr2[0])
                    data['front-camera'].append(arr2[1])
                elif len(arr2)==1 :
                    data['rear-camera'].append(arr2[0])
                    data['front-camera'].append(0)
                else:
                    arr3 = 0
        #data['vendeur'].append('unknow')
        
# casting colors in other to get the real color
couleur_to =list( data['color'])
couleur_ok = []
ram_ok = []
rom_ok = []
taille_ok = []
#print(couleur_to)
ram_to = list(data['ram'])
print(ram_to)
rom_to = list(data['rom'])
taille_to = list(data['screen-size'])
print(int(len(couleur_to)/2))
for i in range(int(len(taille_to)/2)):
    taille_ok.append(taille_to[i*2])
    data['screen-size'] = taille_ok
for i in range(int(len(ram_to)/2)):
    ram_ok.append(ram_to[i*2])
    data['ram'] = ram_ok
for i in range(int(len(rom_to)/2)):
    rom_ok.append(rom_to[i*2])
    data['rom'] = rom_ok
for i in range(int(len(couleur_to)/2)):
    
    #del couleur_to[i*2]
    couleur_ok.append(couleur_to[i*2])
    #ram_ok.append(ram_to[i*2])
    #rom_ok.append(rom_to[i*2])
    #taille_ok.append(taille_to[i*2])
    data['color'] = couleur_ok

#print(data)

#converting the dictionnary to a dataframe
frame = pd.concat([pd.Series(v, name=k) for k, v in data.items()], axis=1)
#print(frame)
# here we export the data to csv file
export_csv = frame.to_csv (r'./dataframe.csv')

Scraping page: 1
Scraping page: 2
Scraping page: 3
Scraping page: 4
Scraping page: 5
Scraping page: 6
Scraping page: 7
Scraping page: 8
Scraping page: 9
Scraping page: 10
Scraping page: 11
Scraping page: 12
Scraping page: 13
Scraping page: 14
Scraping page: 15
Scraping page: 16
Scraping page: 17
Scraping page: 18
Scraping page: 19
Scraping page: 20
Scraping page: 21
Scraping page: 22
Scraping page: 23
Scraping page: 24
Scraping page: 25


# analysing the csv file save


In [3]:
# reading the saved file
df = pd.read_csv('dataframe.csv', index_col=0)
print(df.head())
df.describe()

                                                 nom  vendeur     prix  \
0  Spark Dual SIM - 8Go HDD - 1Go RAM - Noir - 12...   Xtouch  35000.0   
1  A5 Dual SIM - 1Go RAM - 16Go HDD - Noir - 12 Mois   Xtouch  55000.0   
2  Hot 7 Dual SIM - 6,2" - 16 Go HDD - 1 Go RAM -...  Infinix  61500.0   
3  iPhone 5S 32Go HDD - Gris - 12 Mois + Pochette...    Apple  89900.0   
4  Hot 7 Dual Sim - 6,2" - 16 Go HDD - 1 Go RAM - Or  Infinix  61500.0   

   nombre_de_pieces_vendus  ram couleur  taille_ecran   rom  cam_arriere  \
0                     99.0  1.0    noir           5.0   8.0          5.0   
1                     99.0  1.0    noir           5.5  16.0          5.0   
2                   1001.0  1.0    noir           6.2  16.0         13.0   
3                  10001.0  1.0    gris           4.0  32.0          8.0   
4                   1001.0  1.0      or           6.2  16.0         13.0   

   cam_avant  
0        2.0  
1        5.0  
2        8.0  
3        1.2  
4        8.0  


,prix,nombre_de_pieces_vendus,ram,taille_ecran,rom,cam_arriere,cam_avant
count,9.080000e+02,860.000000,784.000000,753.000000,800.000000,629.000000,629.000000
mean,1.896140e+05,2080.204651,8.251276,5.486587,51.840000,11.749285,6.239587
std,1.894133e+05,9359.645386,51.256766,0.793955,94.324842,3.735557,3.995703
min,1.039000e+03,9.000000,1.000000,0.000000,4.000000,0.300000,0.000000
25%,7.875000e+04,49.000000,2.000000,5.000000,16.000000,12.000000,5.000000
50%,1.384940e+05,101.000000,3.000000,5.600000,32.000000,12.000000,5.000000
75%,2.200000e+05,1001.000000,4.000000,6.000000,64.000000,13.000000,8.000000
max,2.210000e+06,100001.000000,512.000000,12.000000,1616.000000,40.000000,24.000000
